In [96]:
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RepeatedStratifiedKFold
from statistics import mean

In [98]:
from sklearn.ensemble import BaggingClassifier

In [93]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import StackingClassifier
import numpy as np
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix

In [103]:
from sklearn.linear_model import RidgeClassifier

In [92]:
from sklearn.metrics import confusion_matrix

In [83]:
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=1) 

In [62]:
df_1 = pd.read_csv('autism_screening.csv')



In [63]:
df_1.head()

,A1_Score,A2_Score,A3_Score,A4_Score,A5_Score,A6_Score,A7_Score,A8_Score,A9_Score,A10_Score,...,gender,ethnicity,jundice,austim,contry_of_res,used_app_before,result,age_desc,relation,Class/ASD
0,1,1,1,1,0,0,1,1,0,0,...,f,White-European,no,no,United States,no,6.0,18 and more,Self,NO
1,1,1,0,1,0,0,0,1,0,1,...,m,Latino,no,yes,Brazil,no,5.0,18 and more,Self,NO
2,1,1,0,1,1,0,1,1,1,1,...,m,Latino,yes,yes,Spain,no,8.0,18 and more,Parent,YES
3,1,1,0,1,0,0,1,1,0,1,...,f,White-European,no,yes,United States,no,6.0,18 and more,Self,NO
4,1,0,0,0,0,0,0,1,0,0,...,f,?,no,no,Egypt,no,2.0,18 and more,?,NO


In [68]:
df_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 702 entries, 0 to 701
Data columns (total 21 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   A1_Score         702 non-null    int64  
 1   A2_Score         702 non-null    int64  
 2   A3_Score         702 non-null    int64  
 3   A4_Score         702 non-null    int64  
 4   A5_Score         702 non-null    int64  
 5   A6_Score         702 non-null    int64  
 6   A7_Score         702 non-null    int64  
 7   A8_Score         702 non-null    int64  
 8   A9_Score         702 non-null    int64  
 9   A10_Score        702 non-null    int64  
 10  age              702 non-null    float64
 11  gender           702 non-null    object 
 12  ethnicity        702 non-null    object 
 13  jundice          702 non-null    object 
 14  austim           702 non-null    object 
 15  contry_of_res    702 non-null    object 
 16  used_app_before  702 non-null    object 
 17  result          

In [69]:
df_1=df_1.dropna().reset_index(drop=True) 

In [70]:
df_1['Class/ASD'] = df_1['Class/ASD'].replace('YES', 1)
df_1['Class/ASD'] = df_1['Class/ASD'].replace('NO', 0)

In [71]:
X = df_1.drop("Class/ASD", axis = 1)   
Y = df_1['Class/ASD']

In [72]:
X = pd.get_dummies(X)


In [73]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.3)

In [74]:
level0 = []
level0.append(('knn', KNeighborsClassifier()))
level0.append(('cart', DecisionTreeClassifier()))
level0.append(('svm', SVC()))
level0.append(('bayes', GaussianNB()))
metamodel = LogisticRegression()
model = StackingClassifier(estimators=level0, final_estimator=metamodel, cv=5)


In [75]:
model.fit(X_train, y_train)

StackingClassifier(cv=5,
                   estimators=[('knn', KNeighborsClassifier()),
                               ('cart', DecisionTreeClassifier()),
                               ('svm', SVC()), ('bayes', GaussianNB())],
                   final_estimator=LogisticRegression())

In [91]:
scores = []
for train_index, test_index in skf.split(X, Y): 
    X_train_fold, X_test_fold = X.values[train_index], X.values[test_index] 
    y_train_fold, y_test_fold = Y.values[train_index], Y.values[test_index] 
    model.fit(X_train_fold, y_train_fold) 
    scores.append(model.score(X_test_fold, y_test_fold))
    
print(scores)


[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
1.0


In [94]:
def evaluate_model(model, X, y):
    cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
    scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
    return scores


In [97]:
print(evaluate_model(model, X_test, y_test ))

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1.]


In [99]:
bag_model = BaggingClassifier(
    base_estimator=DecisionTreeClassifier(), 
    n_estimators=100, 
    max_samples=0.8, 
    bootstrap=True,
    oob_score=True,
    random_state=0
)

In [100]:
bag_model.fit(X_train, y_train)

BaggingClassifier(base_estimator=DecisionTreeClassifier(), max_samples=0.8,
                  n_estimators=100, oob_score=True, random_state=0)

In [102]:
print(evaluate_model(bag_model, X_test, y_test ))

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1.]


In [105]:
Ridge_clf = RidgeClassifier().fit(X_train, y_train)

In [107]:
print(np.mean(evaluate_model(Ridge_clf, X_test, y_test )))

0.9193362193362195
